In [30]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

CHARGEMENT DES DONNÉES

In [31]:
df = pd.read_csv('../data/raw/ai4i2020.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


NETTOYAGE INITIAL

In [32]:
df = df.drop(columns=['UDI', 'Product ID']) 

ENCODAGE DE LA VARIABLE CATÉGORIELLE TYPE

In [33]:
encoder = LabelEncoder()
df['Type'] = encoder.fit_transform(df['Type'])

CRÉATION DE NOUVELLE FEATURES

In [34]:
# Différence de température (La différence entre la température du processus et la température de l'air peut être un indicateur important.)
df["Temp difference"] = df["Process temperature [K]"] - df["Air temperature [K]"]

In [35]:
# Interaction vitesse x couple (cela représente une charge mécanique)
df["Wear speed ratio"] = df["Tool wear [min]"] / (df["Rotational speed [rpm]"] + 1)

VÉRIFICATION DE LA CIBLE

In [36]:
target = 'Machine failure'

In [37]:
df = df.drop(columns=["TWF", "HDF", "PWF", "OSF", "RNF"])

SÉPARATION X / y

In [38]:
X = df.drop(columns=[target])
y = df[target]

TRAIN / Test split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    stratify=y, 
    random_state=42
)

NORMALISATION

In [40]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

SAUVEGARDE DES DONNÉES PRÉPARÉES

In [41]:
processed_train = pd.DataFrame(X_train_scaled, columns=X.columns)
processed_train["Machine failure"] = y_train.values
processed_test = pd.DataFrame(X_test_scaled, columns=X.columns)
processed_test["Machine failure"] = y_test.values   

processed_train.to_csv('../data/processed/processed_train.csv', index=False)
processed_test.to_csv('../data/processed/processed_test.csv', index=False)